# Mounting Drive 

### Connecting to our group folders data

In [ ]:
# This is to help connect this jupyter notebook to our data that is in google drive

from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
# Importing the necessary libraries that you need
import os
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [ ]:
# This will change the directory straight to the google drive "DATA" folder
PATH = 'drive/MyDrive/Credit Line Increase Competition21/data'

data_paths = os.listdir(PATH)
os.chdir(PATH)

data_paths

['ucf_unsecured_debt.csv',
 'ucf_PSCU_UCF_2021_Data.csv',
 '20211104_APPEND_ucf_rates_history.csv',
 '20211104_APPEND_ucf_bureau_score_history.csv',
 '2021_2022_ucf_field_definitions.docx',
 'full_df_angelo.csv',
 'no_process.csv',
 'full_df.csv',
 'Features Used.png']

In [ ]:
def missing_zero_values_table(df):
  """ Will find NA and zero value statistics for dataframe
  
  :df: any pandas dataframe
  
  """
  zero_val = (df == 0.00).astype(int).sum(axis=0)
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
  mz_table = mz_table.rename(
  columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
  mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
  mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
  mz_table['Data Type'] = df.dtypes
  mz_table = mz_table[mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " 
               + str(df.shape[0]) + " Rows.\nThere are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
  return mz_table

def combine_tables(unsecured, credit_scores, pscu, rates):
  """ Will run our full preprocessing procedure and join the given tables

  :unsecured_path: 'ucf_unsecured_debt.csv'
  :pscu_path: 'ucf_PSCU_UCF_2021_Data.csv'
  :credit_score_path: '20211104_APPEND_ucf_bureau_score_history.csv'
  :rates_path: '20211104_APPEND_ucf_rates_history.csv'

  """

  # Obtaining each user's most recent rates
  rates = rates.sort_values(['ProcessDate', 'credit_line_Rate'], ascending=False).groupby('ucfID').first().drop(['ProcessDate'], axis = 1)
  rates['credit_line_Rate'] /= 1000

  # Unsecured table pre-processing, taking sum of each 
  unsecured = unsecured.drop_duplicates()
  unsecured = unsecured.fillna(0)
  unsecured = unsecured.groupby('ucfID').sum()

  # Credit Score History table pre-processing
  credit_scores.drop_duplicates()
  credit_scores['credit_bureau_score_date'] = pd.to_datetime(credit_scores['credit_bureau_score_date'])
  credit_scores = credit_scores[credit_scores['credit_bureau_score_date'] >= '2016-09-30']

  most_recent_score = pd.merge(pd.DataFrame(credit_scores.groupby('ucfID')['credit_bureau_score_date'].max()), 
                               credit_scores, on = ['ucfID', 'credit_bureau_score_date'])
  
  most_recent_score = pd.merge(np.round(credit_scores.groupby('ucfID').mean()), most_recent_score, on = 'ucfID')
  most_recent_score.rename(columns = {'credit_bureau_score_x': 'recent_cred_score',
                                      'credit_bureau_score_y': 'avg_score_5_yrs'},
                           inplace = True)
  most_recent_score.drop('credit_bureau_score_date', axis = 1, inplace = True)

  # Changing all dates to number of days, beginning from start of competition
  ################################################################################
  pscu = pscu.drop_duplicates()

  dates = ['card_open_date', 'credit_line_last_change_date', 'card_expiration_date',
            'credit_bureau_score_date', 'last_purchase_date', 'last_cash_advance_date',
            'last_payment_date', 'last_transaction_date', 'card_account_transfer_upgrade_date']

  pscu[dates] = pscu[dates].apply(pd.to_datetime)

  current_time = datetime.strptime('2021/09/15 00:00:00', '%Y/%m/%d %H:%M:%S')

  for i in dates:
    pscu[i] = (current_time - pscu[i]).dt.days

  for i in dates:
    pscu.loc[pscu[i] == 18884, i] = 0

  pscu = pscu.groupby('ucfID').max()

  # PSCU table and joining the tables altogether
  # Take max value of each column to combat repeated ucfID
  print("The shape of the df before joining with the unsecured table:")
  print(pscu.shape)
  general = pd.merge(pscu, unsecured, on = 'ucfID', how = 'inner')
  print("\nThe shape of the df is now:")
  print(general.shape)
  print("\nNow joining with credit scores table:")
  general = pd.merge(general, most_recent_score, on = 'ucfID', how = 'inner')
  print("\nThe shape of the df is now:")
  print(general.shape)
  print("\nNow joining with rates table:")
  general = pd.merge(general, rates, on = 'ucfID', how = 'inner')
  print("\nThe shape of the df after joining all tables is:")
  print(general.shape)

  return general

def preprocess_df(combined_df):
  """ Will preprocess our DataFrame and Produce Transformed Variables

  :combined_df: all tables combined
  
  DATA PREPROCESSING STEPS:
  1. TRANSFORM: re-summing lifetime_delinquincy, turn negative balance to 0
  ['lifetime_delinquent_cycle_count', 'year_to_date_average_daily_balance_amount'] 

  2. REMOVE: columns with 100% null values and 100% 0 values
  ['predictive_growth_score', 'predictive_hardship_score', 
   'predictive_hardship_score_date', 'predictive_attrition_score', 
   'predictive_attrition_score_date', 'primary_cardholder_debt_to_income_ratio', 
   'predictive_growth_score_date', 'unpaid_dispute_amount']

  3.REMOVE: Columns that had errors in start of competition
  ['credit_bureau_score', 'primary_cardholder_personal_monthly_income', 
   'primary_cardholder_disposable_monthly_income', 'credit_bureau_score_date']

  4. CREATE: Binarizing Overlimits 1 = Yes Overlimit, 0 = No Overlimit
  ['overlimit_binary']

  5. REMOVE: Credit Line Amounts above $30,0000 (based off Addition giving max $30k)
  ['credit_line_amount']

  6. CREATE: Binarizing of Cash Advance in 3 years
  ['cash_advance_3_yrs']

  7. CREATE: Binarizing of Cash Advance in 7 years
  ['cash_advance_7_yrs']

  8. CREATE: Average annual historical disputes, n_disputes/card_open_date
  ['avg_annual_historical_dispute']

  9. CREATE: Average annual 1 and 2 cycle delinquincies, n_delinquincies/card_open 
  ['avg_annual_delinquent_1_cycle', 'avg_annual_delinquent_2_cycle']

  10. REMOVE: Abnormal values in 2nd cycle disputes
  ['avg_annual_delinquent_2_cycle']

  11. REMOVE: Abnormally high credit card lengths of credit > 48 years
  ['card_open_date']

  12. TRANSFORM: Cards 15 days expired, will just change to 0
  ['card_expiration_date']

  13. TRANSFORM: Binarizing returned check for year, 1 = Returned, 0 = No Returns
  ['year_to_date_return_check_count']
 
  14. CREATE: Percentage credit used (cred_line_amt - avail_cred)/(cred_line_amt)
  ['perc_credit_used']

  15. CREATE: Percentage Change of most recent cred_score/avg_5_yrs_cred_score
  ['perc_change_credit_score']

  16. TRANSFORM: Binarizing user's last purchase date in past 365 day 
  ['last_purchase_date']

  17. REMOVE: More columns that may complicate analysis
  ['year_to_date_overlimit_months_count', 'lifetime_delinquent_cycle_count',
   'year_to_date_high_balance_amount', 'previous_year_high_balance_amount', 
   'lifetime_high_balance_amount', 'card_account_transfer_upgrade_date', 
   'available_credit_amount', 'current_balance_amount', 'card_expiration_date',
   'last_payment_date']

  """

  # 1
  combined_df = combined_df.drop_duplicates()
  combined_df['lifetime_delinquent_cycle_count'] = combined_df['lifetime_delinquent_1_cycle_count'] + combined_df['lifetime_delinquent_2_cycles_count']
  combined_df.loc[combined_df['year_to_date_average_daily_balance_amount'] < 0, 'year_to_date_average_daily_balance_amount'] = 0
  
  # 2
  null_columns = ['predictive_growth_score', 'predictive_hardship_score', 'predictive_hardship_score_date', 
                  'predictive_attrition_score', 'predictive_attrition_score_date',
                  'primary_cardholder_debt_to_income_ratio', 'predictive_growth_score_date', 
                  'unpaid_dispute_amount']
  combined_df = combined_df.drop(null_columns, axis = 1)
  combined_df = combined_df.fillna(0)
  
  # 3
  errors = ['credit_bureau_score', 'primary_cardholder_personal_monthly_income', 
            'primary_cardholder_disposable_monthly_income', 'credit_bureau_score_date']
  combined_df = combined_df.drop(errors, axis = 1)

  # 4
  combined_df['overlimit_binary'] = combined_df['year_to_date_overlimit_months_count']
  combined_df.loc[combined_df['overlimit_binary'] > 0, 'overlimit_binary'] = 1

  # 5
  combined_df = combined_df[combined_df['credit_line_amount'] < 30000]

  # 6
  combined_df['cash_advance_3_yrs'] = combined_df['last_cash_advance_date']
  combined_df.loc[combined_df['last_cash_advance_date'] <= 365*3, 'cash_advance_3_yrs'] = 1
  combined_df.loc[combined_df['last_cash_advance_date'] > 365*3, 'cash_advance_3_yrs'] = 0

  # 7
  combined_df['cash_advance_7_yrs'] = combined_df['last_cash_advance_date']
  combined_df.loc[combined_df['last_cash_advance_date'] <= 365*7, 'cash_advance_7_yrs'] = 1
  combined_df.loc[combined_df['last_cash_advance_date'] > 365*7, 'cash_advance_7_yrs'] = 0

  # 8
  combined_df['avg_annual_historical_dispute'] = combined_df['historical_dispute_count']/combined_df['card_open_date']*365
  
  # 9
  combined_df['avg_annual_delinquent_1_cycle'] = (combined_df['lifetime_delinquent_1_cycle_count'])/combined_df['card_open_date']*365
  combined_df['avg_annual_delinquent_2_cycle'] = (combined_df['lifetime_delinquent_2_cycles_count'])/combined_df['card_open_date']*365
  
  # 10
  combined_df = combined_df[combined_df['avg_annual_delinquent_2_cycle'] < combined_df['avg_annual_delinquent_2_cycle'].quantile(.99)]
  
  # 11
  combined_df = combined_df[combined_df['card_open_date'] < 17500]
  
  # 12
  combined_df.loc[combined_df['card_expiration_date'] > 0, 'card_expiration_date'] = 0
  
  # 13 
  combined_df.loc[combined_df['year_to_date_return_check_count'] > 0, 'year_to_date_return_check_count'] = 1
  
  # 14
  combined_df['avg_perc_credit_used'] = combined_df['year_to_date_average_daily_balance_amount']/combined_df['credit_line_amount']
  
  # 15
  combined_df['perc_change_credit_score'] = (combined_df['recent_cred_score'] - combined_df['avg_score_5_yrs']) / combined_df['avg_score_5_yrs'] * 100

  # 16 
  combined_df.loc[combined_df['last_purchase_date'] < 365, 'last_purchase_date'] = 1

  # 17 
  more_removals = ['lifetime_delinquent_cycle_count', 'year_to_date_high_balance_amount', 
                   'previous_year_high_balance_amount', 'lifetime_high_balance_amount', 
                   'card_account_transfer_upgrade_date', 'year_to_date_overlimit_months_count', 
                   'available_credit_amount', 'current_balance_amount', 'card_expiration_date',
                   'last_payment_date']
  combined_df = combined_df.drop(more_removals, axis = 1)

  return combined_df.fillna(0)


# Reading in each data table

Tried to include this in the analysis, we thought the information would be important, so we grouped by ID and added the sum of each column. There were many users that were unaccounted for and had 0 credit cards, 0 loans.

# EDA on the General PSCU Table

Removing columns where 100% are null and 0's

In [ ]:
general = pd.read_csv('ucf_PSCU_UCF_2021_Data.csv')

In [ ]:
general_na_table = missing_zero_values_table(general)
general_na_table.head(15)

Your selected dataframe has 44 columns and 351835 Rows.
There are 31 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
predictive_growth_score_date,0,351835,100.0,351835,100.0,float64
primary_cardholder_debt_to_income_ratio,0,351835,100.0,351835,100.0,float64
predictive_hardship_score,0,351835,100.0,351835,100.0,float64
predictive_hardship_score_date,0,351835,100.0,351835,100.0,float64
predictive_attrition_score,0,351835,100.0,351835,100.0,float64
predictive_attrition_score_date,0,351835,100.0,351835,100.0,float64
predictive_growth_score,0,351835,100.0,351835,100.0,float64
previous_year_unpaid_billed_interest_amount,18712,320418,91.1,339130,96.4,float64
current_balance_amount,10423,320418,91.1,330841,94.0,float64
year_to_date_unpaid_billed_interest_amount,20076,320418,91.1,340494,96.8,float64


# Processing General Columns

In [ ]:
general_df = pd.read_csv('ucf_PSCU_UCF_2021_Data.csv', index_col=0)
rates_df = pd.read_csv('20211104_APPEND_ucf_rates_history.csv')
scores_df = pd.read_csv('20211104_APPEND_ucf_bureau_score_history.csv')
unsecured_df = pd.read_csv('ucf_unsecured_debt.csv')

In [ ]:
combined_df = combine_tables(unsecured_df, scores_df, general_df, rates_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

The shape of the df before joining with the unsecured table:
(30533, 42)

The shape of the df is now:
(30405, 47)

Now joining with credit scores table:

The shape of the df is now:
(22669, 50)

Now joining with rates table:

The shape of the df after joining all tables is:
(22592, 51)


In [ ]:
combined_df.head()

,ucfID,lifetime_delinquent_cycle_count,lifetime_delinquent_1_cycle_count,lifetime_delinquent_2_cycles_count,card_open_date,credit_line_last_change_date,card_expiration_date,credit_bureau_score,credit_bureau_score_date,primary_cardholder_debt_to_income_ratio,year_to_date_high_balance_amount,previous_year_high_balance_amount,lifetime_high_balance_amount,last_purchase_date,last_cash_advance_date,last_transaction_date,card_account_transfer_upgrade_date,predictive_hardship_score,predictive_hardship_score_date,predictive_attrition_score,predictive_attrition_score_date,predictive_growth_score,predictive_growth_score_date,last_payment_date,year_to_date_overlimit_months_count,year_to_date_return_check_count,primary_cardholder_personal_monthly_income,primary_cardholder_disposable_monthly_income,year_to_date_cash_advance_count,year_to_date_cash_advance_amount,year_to_date_purchase_net_count,year_to_date_purchase_net_amount,year_to_date_average_daily_balance_amount,year_to_date_unpaid_billed_interest_amount,current_balance_amount,credit_line_amount,year_to_date_purchase_interest_amount,previous_year_reportable_interest_amount,previous_year_unpaid_billed_interest_amount,available_credit_amount,purchase_transaction_net_count,historical_dispute_count,unpaid_dispute_amount,n_loans,sumLoanDebt,n_credit_cards,sumCreditLimit,tot_Unsec_Debt,recent_cred_score,avg_score_5_yrs,credit_line_Rate
0,000156E1-0152-4D92-B1D9-6D37B24763C9,3,3,0,2779,1049.0,-350,720,91.0,NaN,568,0,6001.66,57.0,2235.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,0,0,11.00,11.00,24.0,6664.49,17.0,696.95,395.18,0.00,422.50,26000.0,12.55,0.00,0.00,25577.50,34.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,769.0,720,8.75
1,0001B18E-3807-46AC-943C-37C1B968BBD5,2,2,0,1589,1598.0,-288,648,91.0,NaN,6613,6847,8006.36,52.0,1584.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,0,0,17.75,17.75,47.0,11266.71,17.0,754.70,6481.75,71.91,6374.37,8000.0,146.82,858.10,76.84,1625.63,106.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,624.0,648,17.75
2,0003AD69-81F6-4CE2-8620-D233E2D5C8D4,1,1,0,2911,76.0,-380,727,91.0,NaN,8298,14675,15451.02,1293.0,559.0,559.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190.0,0,0,16.50,16.50,101.0,75694.24,0.0,0.00,8072.94,15.84,0.00,10000.0,0.00,228.26,6.77,10000.00,467.0,0.0,0.0,2.0,23831.67,0.0,0.0,23831.67,686.0,727,9.75
3,0006A937-F473-450F-87F3-0DF09EE6B3ED,0,0,0,1948,1049.0,-1293,812,91.0,NaN,0,1970,1969.69,635.0,1072.0,636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,0,0,11.00,11.00,43.0,13075.94,0.0,0.00,0.00,0.00,0.00,4500.0,0.00,47.65,15.04,4500.00,232.0,0.0,0.0,0.0,0.00,1.0,4500.0,4500.00,807.0,812,8.75
4,000917CA-9984-40F5-85B4-2718686AE4C2,0,0,0,3150,532.0,-319,784,91.0,NaN,1200,250,3539.85,51.0,3138.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,0,0,16.50,16.50,76.0,11598.73,52.0,1600.02,220.20,0.00,137.60,8000.0,0.00,0.00,0.00,8000.00,214.0,0.0,0.0,0.0,0.00,1.0,5000.0,5000.00,704.0,784,14.25


In [ ]:
full_df = preprocess_df(combined_df)
print(full_df.shape)
full_df.head()

(22222, 37)


,ucfID,lifetime_delinquent_1_cycle_count,lifetime_delinquent_2_cycles_count,card_open_date,credit_line_last_change_date,last_purchase_date,last_cash_advance_date,last_transaction_date,year_to_date_return_check_count,year_to_date_cash_advance_count,year_to_date_cash_advance_amount,year_to_date_purchase_net_count,year_to_date_purchase_net_amount,year_to_date_average_daily_balance_amount,year_to_date_unpaid_billed_interest_amount,credit_line_amount,year_to_date_purchase_interest_amount,previous_year_reportable_interest_amount,previous_year_unpaid_billed_interest_amount,purchase_transaction_net_count,historical_dispute_count,n_loans,sumLoanDebt,n_credit_cards,sumCreditLimit,tot_Unsec_Debt,recent_cred_score,avg_score_5_yrs,credit_line_Rate,overlimit_binary,cash_advance_3_yrs,cash_advance_7_yrs,avg_annual_historical_dispute,avg_annual_delinquent_1_cycle,avg_annual_delinquent_2_cycle,avg_perc_credit_used,perc_change_credit_score
0,000156E1-0152-4D92-B1D9-6D37B24763C9,3,0,2779,1049.0,1.0,2235.0,58.0,0,24.0,6664.49,17.0,696.95,395.18,0.00,26000.0,12.55,0.00,0.00,34.0,0.0,0.0,0.00,0.0,0.0,0.00,769.0,720,8.75,0,0.0,1.0,0.0,0.394027,0.0,0.015199,6.805556
1,0001B18E-3807-46AC-943C-37C1B968BBD5,2,0,1589,1598.0,1.0,1584.0,54.0,0,47.0,11266.71,17.0,754.70,6481.75,71.91,8000.0,146.82,858.10,76.84,106.0,0.0,0.0,0.00,0.0,0.0,0.00,624.0,648,17.75,0,0.0,1.0,0.0,0.459408,0.0,0.810219,-3.703704
2,0003AD69-81F6-4CE2-8620-D233E2D5C8D4,1,0,2911,76.0,1293.0,559.0,559.0,0,101.0,75694.24,0.0,0.00,8072.94,15.84,10000.0,0.00,228.26,6.77,467.0,0.0,2.0,23831.67,0.0,0.0,23831.67,686.0,727,9.75,0,1.0,1.0,0.0,0.125386,0.0,0.807294,-5.639615
3,0006A937-F473-450F-87F3-0DF09EE6B3ED,0,0,1948,1049.0,635.0,1072.0,636.0,0,43.0,13075.94,0.0,0.00,0.00,0.00,4500.0,0.00,47.65,15.04,232.0,0.0,0.0,0.00,1.0,4500.0,4500.00,807.0,812,8.75,0,1.0,1.0,0.0,0.000000,0.0,0.000000,-0.615764
4,000917CA-9984-40F5-85B4-2718686AE4C2,0,0,3150,532.0,1.0,3138.0,52.0,0,76.0,11598.73,52.0,1600.02,220.20,0.00,8000.0,0.00,0.00,0.00,214.0,0.0,0.0,0.00,1.0,5000.0,5000.00,704.0,784,14.25,0,0.0,0.0,0.0,0.000000,0.0,0.027525,-10.204082


# Saving Processed files

In [ ]:
full_df.to_csv('full_df.csv')